In [2]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# QA Bot Pipeline\n",
    "This notebook demonstrates the entire pipeline for a Retrieval-Augmented Generation (RAG) model for a Question Answering (QA) bot.\n",
    "The bot processes uploaded documents, generates embeddings, and retrieves answers based on user queries."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Install required libraries\n",
    "!pip install pinecone-client cohere PyPDF2 python-dotenv"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import necessary libraries\n",
    "import os\n",
    "import pinecone\n",
    "import cohere\n",
    "from PyPDF2 import PdfReader\n",
    "from dotenv import load_dotenv\n",
    "\n",
    "# Load environment variables\n",
    "load_dotenv()\n",
    "PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')\n",
    "COHERE_API_KEY = os.getenv('COHERE_API_KEY')\n",
    "\n",
    "# Initialize Pinecone and Cohere clients\n",
    "pinecone.init(api_key=PINECONE_API_KEY)\n",
    "cohere_client = cohere.Client(api_key=COHERE_API_KEY)\n",
    "\n",
    "# Constants\n",
    "INDEX_NAME = 'your-index-name'  # Change to your index name"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [],
   "source": [
    "def extract_text_from_pdf(file_path):\n",
    "    \"\"\"\n",
    "    Extract text from a PDF file.\n",
    "    \"\"\"\n",
    "    with open(file_path, 'rb') as f:\n",
    "        pdf_reader = PdfReader(f)\n",
    "        text = \"\"\n",
    "        for page in pdf_reader.pages:\n",
    "            text += page.extract_text() or \"\"\n",
    "    return text"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {},
   "outputs": [],
   "source": [
    "def split_text_into_chunks(text, chunk_size=500):\n",
    "    \"\"\"\n",
    "    Split the text into manageable chunks.\n",
    "    \"\"\"\n",
    "    return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "metadata": {},
   "outputs": [],
   "source": [
    "def store_embeddings_in_pinecone(chunks):\n",
    "    \"\"\"\n",
    "    Store text embeddings in Pinecone.\n",
    "    \"\"\"\n",
    "    # Ensure the index exists\n",
    "    if INDEX_NAME not in pinecone.list_indexes():\n",
    "        pinecone.create_index(INDEX_NAME)\n",
    "    index = pinecone.Index(INDEX_NAME)\n",
    "    \n",
    "    # Generate embeddings using Cohere\n",
    "    embeddings = cohere_client.embed(texts=chunks).embeddings\n",
    "    \n",
    "    # Prepare data for Pinecone\n",
    "    vectors = [(str(i), embeddings[i]) for i in range(len(embeddings))]\n",
    "    \n",
    "    # Upsert vectors into Pinecone\n",
    "    index.upsert(vectors=vectors)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "metadata": {},
   "outputs": [],
   "source": [
    "def retrieve_answer(question):\n",
    "    \"\"\"\n",
    "    Retrieve the answer to a question based on the stored embeddings in Pinecone.\n",
    "    \"\"\"\n",
    "    # Get embedding for the question\n",
    "    question_embedding = cohere_client.embed(texts=[question]).embeddings[0]\n",
    "    \n",
    "    # Query Pinecone for the most relevant documents\n",
    "    index = pinecone.Index(INDEX_NAME)\n",
    "    query_response = index.query(queries=[question_embedding], top_k=3)\n",
    "    \n",
    "    # Generate answer using the relevant chunks\n",
    "    relevant_chunks = [match['id'] for match in query_response['matches']]\n",
    "    answer = cohere_client.generate(\n",
    "        prompt=f\"Answer the question '{question}' using the following context: {relevant_chunks}\",\n",
    "        max_tokens=100,\n",
    "        temperature=0.5\n",
    "    ).generations[0].text.strip()\n",
    "    \n",
    "    return answer"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Example usage\n",
    "# Load a PDF file and process it\n",
    "file_path = 'path_to_your_document.pdf'  # Change to your document path\n",
    "text = extract_text_from_pdf(file_path)\n",
    "chunks = split_text_into_chunks(text)\n",
    "store_embeddings_in_pinecone(chunks)\n",
    "\n",
    "# Ask a question\n",
    "question = 'What is the main topic of the document?'\n",
    "answer = retrieve_answer(question)\n",
    "print(f'Question: {question}')\n",
    "print(f'Answer: {answer}')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Conclusion\n",
    "This notebook provides a complete pipeline for a QA bot using Retrieval-Augmented Generation. It demonstrates how to process documents, generate embeddings, and retrieve answers using a vector database."
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}

{'cells': [{'cell_type': 'markdown',
   'metadata': {},
   'source': ['# QA Bot Pipeline\n',
    'This notebook demonstrates the entire pipeline for a Retrieval-Augmented Generation (RAG) model for a Question Answering (QA) bot.\n',
    'The bot processes uploaded documents, generates embeddings, and retrieves answers based on user queries.']},
  {'cell_type': 'code',
   'execution_count': 1,
   'metadata': {},
   'outputs': [],
   'source': ['# Install required libraries\n',
    '!pip install pinecone-client cohere PyPDF2 python-dotenv']},
  {'cell_type': 'code',
   'execution_count': 2,
   'metadata': {},
   'outputs': [],
   'source': ['# Import necessary libraries\n',
    'import os\n',
    'import pinecone\n',
    'import cohere\n',
    'from PyPDF2 import PdfReader\n',
    'from dotenv import load_dotenv\n',
    '\n',
    '# Load environment variables\n',
    'load_dotenv()\n',
    "PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')\n",
    "COHERE_API_KEY = os.getenv('COHERE_API_K